# Szyfrowanie homomorficzne

Szyfrowanie homomorficzne to rodzaj szyfrowania z kluczem publicznym, w którym **deformowalność szyfrogramów** to pożądana cecha kryptosystemu. Przepomnijmy, że deformowalność szyfrogramów to własność polegająca na możliwości modyfikacji szyfrogramów w taki sposób, że po odszyfrowaniu tak zmodyfikowanego szyfrogramu otrzymujemy tekst jawny, który jest **sensownie** powiązany z oryginalnym tekstem jawnym.

Innymi słowy, szyfrowanie homomorficzne umożliwia wykonywanie operacji (matematycznych) na zaszyfrowanych danych, tak by pod odszyfrowaniu otrzymać wynik tak samo poprawny jak w sytuacji gdy operacją wykonywane na jawnych danych. 

Rozróżniamy kilka poziomów szyfrowania homomorficznego:
1. Szyfrowanie częściowo homomorficzne - kryptosystem wspiera tylko jedną operację
2. _Somewhat Homomorphic Encryption_ - kryptosystem wspiera obie operacje, ale sekwencja możliwych operacji jest ograniczona jeśli chodzi o długość.
3. Szyfrowanie w pełni homomorficzne (FHE - _Fully Homomorphic Encryption_) - pełna obsługa bez ograniczeń operacji na zaszyfrowanych danych. 

Szyfrowanie homomorficzne jest szczególnie pożądane w sytuacji gdy powierzamy swoje dane do przetwarzania obcemu podmiotowi.

In [1]:
## Funkcje pomocnicze
def gcd(a, b):
    # GCD - Greatest Common Divisor, Największy wspólny dzielnik
    while a != 0:
        a, b = b % a, a
    return b

def lcm(a, b):
    # LCM - Lowest Common Multiple, Najmniejsza wspólna wielokrotność
    return a * b // gcd(a, b)

def findModInverse(a, m):
    # Zwraca liczbę x odwrotną do a ciele skończonym modulo m
    # czyli (a*x) % m =1

    if gcd(a, m) != 1:
        return None #a i m muszą być względnie pierwsze aby istniał element odwrotny

    # Rozszerzony algorytm Euklidesa
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # // operator dzielenie całkowitoliczbowego
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m

### Zadanie 1
Dziełając wyłącznie na szyfrogramach RSA zaproponuj i zaimplementuje metodą fałszującą odszyfrowaną wiadomość tak by była ona liczą podzielną przez 17.  

In [4]:
def generateRSAKeys(keySize):
    print('1. Generujemy liczby p i q')
    p=49109
    q=40639
    e = 65537
    print('3. Obliczamy wykładnik prywatny: odwrotność e modulo (p-1)(q-1c)')
    d=findModInverse(e, (p-1)*(q-1))
    n= p*q
    publicKey = (n, e)
    privateKey = (n, d)
    print('Klucz publiczny:', publicKey)
    print('Klucz prywatny:', privateKey)
    return (publicKey, privateKey)

public, private = generateRSAKeys(16)

print('Generujemy klucze publiczny i prywatny:', public, private)

def forge_ciphertext(ciphertext, public_key, k=17):
    n, e = public_key
    forged_ciphertext = (ciphertext * pow(k, e, n)) % n
    return forged_ciphertext

def encryptRSA(data_number, modulus, exp):
    data_encrypted = pow(data_number, exp, modulus)
    return data_encrypted

def decryptRSA(data_number, modulus, exp):
    data_decrypted = pow(data_number, exp, modulus)
    return data_decrypted

message = 12345

cipher = encryptRSA(message, public[0], public[1])
forged_cipher = forge_ciphertext(cipher, public, 17)
decrypted_forged = decryptRSA(forged_cipher, private[0], private[1])

print("Oryginalna wiadomość:", message)
print("Odszyfrowana sfałszowana wiadomość:", decrypted_forged)
print("Podzielna przez 17:", decrypted_forged % 17 == 0)

1. Generujemy liczby p i q
3. Obliczamy wykładnik prywatny: odwrotność e modulo (p-1)(q-1c)
Klucz publiczny: (1995740651, 65537)
Klucz prywatny: (1995740651, 601158737)
Generujemy klucze publiczny i prywatny: (1995740651, 65537) (1995740651, 601158737)
Oryginalna wiadomość: 12345
Odszyfrowana sfałszowana wiadomość: 209865
Podzielna przez 17: True


## Kryptosystem Pailliera

W 1999 roku Pascal Paillier zaproponował kryptosystem, którego bezpieczeństwo oparte było o problem faktoryzacji liczb całkowitych i problem logarytmu dyskretnego.


#### Generowanie kluczy (wersja podstawowa)

1. Wybierz losowo dwie duże liczby pierwsze $p$ i $q$ tak aby $gcd(pq, (p-1)(q-1)) = 1$.
2. Oblicz $n = pq$
3. Oblicz $\lambda = lcm(p-1,q-1)$ ($lcm$ --- Least Common Multiple, Najmniejsza Wspólna Wielokrotność)
4. Zdefiniuj funkcje $L(x) = \frac{x-1}{n}$
5. Wybierz losowo $g \in Z^*_{n^2}$ (liczba całkowita w zakresie 1 do $n^2$)
6. Oblicz odwrotność multiplikatywną $\mu = L(g^\lambda \bmod n^2))^{-1} \bmod n$. Jeśli $\mu$ nie istnieje zacznij od nowa.

Klucz publiczny ma postać: $pk = (n,g)$
Klucz prywatny ma postać: $sk = (\lambda, \mu)$

#### Generowanie kluczy wersja uproszczona
Jeśli $p$ i $q$ są podobniej długości można użyć prostszego wariantu:
1. $g = n+1$
2. $\lambda = \phi(n)$
3. $\mu = \phi(n)^{-1} \bmod n$

#### Szyfrowanie

1. Tekstem jawnym jest liczba $m$ zakresu $0 \le m < n$.

2. Wybierz losową liczbę z zakresu $0 \le r < n$ oraz względnie pierwszą z $n$

3. Oblicz szyfrogram $c= g^m \cdot r^n \bmod n^2$

#### Deszyfrowanie

1. Szyfrogram musi być liczbą z zakresu $0 < c < n^2$
2. Oblicz tekst jawny $m= L(c^\lambda \bmod n^2)\cdot \mu \bmod n$

### Homomorficzne własności schematu Pailliera

1. Dodawanie dwóch liczb:
$$D_{priv}(E_{pub}(m_1) \cdot E_{pub}(m_2) \bmod n^2)= m_1 + m_2 \bmod n$$

2. Mnożenie szyfrogramu przez liczbę:
$$D_{priv}(E_{pub}(m_1)^{m_2} \bmod n^2)= m_1 \cdot m_2 \bmod n$$

### Zadanie 2
Zaimplementuj szyfrowanie i deszyfrowanie Pailliera oraz funkcje umożliwiające homomorficzne operacja dodawania oraz mnożenia przez liczbę całkowitą. Wykaż poprawność operacji homomorficznych.

In [5]:
def genPaillierKeys():
    p = 49109
    q = 40639
    nt = p*q
    gt = nt+1

    lambdat = lcm(p-1, q-1)
    mut = findModInverse((pow(gt, lambdat, nt*nt)-1)//nt, nt)

    return nt, gt, lambdat, mut

n, g, Lambda, Mu = genPaillierKeys()

def encryptPaillier(m, n, g):
    r = 12345
    c = (pow(g, m, n*n) * pow(r, n, n*n)) % (n*n)
    return c

def decryptPaillier(c, Lambda, Mu, n):
    x = pow(c, Lambda, n*n)
    L = (x - 1) // n
    m = (L * Mu) % n
    return m

# Testy
cipher_paillier = encryptPaillier(12345, n, g)
plain_paillier = decryptPaillier(cipher_paillier, Lambda, Mu, n)
print("Paillier Decrypted:", plain_paillier)

Paillier Decrypted: 12345


## Szyfrowanie Homomorficzne

Rozważmy następujący naiwny kryptosystem :

#### Generowanie kluczy 
1. wybierz dużą liczbę nieparzystą $p$
2. wybierz ogranicznie szumu, znacznie mniejsze od $p$: $B << p$

#### Szyfrowanie 
1. szyfrujemy wiadomości 1-bitowe: $m$
2. wylosuj duże $q$ i $r \in \{0,...,B\}$
3. oblicz $c=m+2r + pq$

#### Deszyfrowanie
1. oblicz $t=c \bmod p$
2. oblicz $t \bmod 2$

#### Operacje homomorficzne (w ciele GF(2)):
1. Dodawania:$c_+ = c_1 + c_2 (\bmod 2)$
2. Mnożenie: $c_* = c_1 \cdot c_2 (\bmod 2)$


### Zadanie 3
Zaimplementuj opisany powyżej trywialny kryptosystem. Sprawdź poprawność działania dla obu operacji. Możesz zastosować następujące parametry:
- $p = 29$
- $q \approx 22$
- $r = 3$

Sprawdź jak będzie dział ten kryptosystem gdy dodasz do siebie trzy liczby jedna po drugiej np.: $c_1(0) + c_2(0) +c_3(0)$

In [28]:
p = 29
q = 22
r = 3

def encrypt(m):
    c = m + 2*r + p*q
    return c

def decrypt(c):
    return (c % p) % 2

m = 0
c = encrypt(m)

print("Wartość c:", c)

t = decrypt(c)

print("Wartość t:", t)

m1 = 1
m2 = 1
m3 = 1

c1 = encrypt(m1)
c2 = encrypt(m2)
c3 = encrypt(m3)

c_sum = (c1 + c2 + c3)
print("Suma c mod 2:", decrypt(c_sum))

c_mult = (c1 * c2 * c3) % 2
print("Iloczyn c mod 2:", decrypt(c_mult))

Wartość c: 644
Wartość t: 0
Suma c mod 2: 1
Iloczyn c mod 2: 1


## Szyfrowanie w Pełni Homomorficzne *Fully Homomorphic Encryption*
Szyfrowanie w Pełni Homomorficzne umożliwia realizację wszystkicj operacji arytmetycznych czyli dodawania i mnożenia.

Wykorzystaj bibliotekę Pyfhel w celu sprawdzenia możliwości realizacji operacji matematycznych pod osłoną szyfrowania homomorficznego.

https://pyfhel.readthedocs.io/en/latest/

In [29]:
# !pip install pyfhel
import numpy as np
from Pyfhel import Pyfhel, PyPtxt, PyCtxt

HE = Pyfhel()
HE.contextGen(scheme='bfv', n=2**14, t_bits=20)
HE.keyGen()

int1 = np.array([127])
int2 = np.array([-2])
ciphertext1 = HE.encryptInt(int1)
ciphertext2 = HE.encryptInt(int2)


ciphertextSum = ciphertext1 + ciphertext2
summ = HE.decryptInt(ciphertextSum)
print(f"Suma = {summ}")

Suma = [125   0   0 ...   0   0   0]


In [30]:

HE = Pyfhel()           # Creating empty Pyfhel object
ckks_params = {
    'scheme': 'CKKS',   # can also be 'ckks'
    'n': 2**14,         # Polynomial modulus degree. For CKKS, n/2 values can be
                        #  encoded in a single ciphertext.
                        #  Typ. 2^D for D in [10, 15]
    'scale': 2**30,     # All the encodings will use it for float->fixed point
                        #  conversion: x_fix = round(x_float * scale)
                        #  You can use this as default scale or use a different
                        #  scale on each operation (set in HE.encryptFrac)
    'qi_sizes': [60, 30, 30, 30, 60] # Number of bits of each prime in the chain.
                        # Intermediate values should be  close to log2(scale)
                        # for each operation, to have small rounding errors.
}
HE.contextGen(**ckks_params)  # Generate context for ckks scheme
HE.keyGen()             # Key Generation: generates a pair of public/secret keys
HE.rotateKeyGen()


float1 = np.array([1.2])
float2 = np.array([2.2])
ciphertext1 = HE.encryptFrac(float1)
ciphertext2 = HE.encryptFrac(float2)

ciphertextSum = ciphertext1 + ciphertext2
summ = HE.decryptFrac(ciphertextSum)
print(f"Suma = {summ}")

Suma = [ 3.39999937e+00 -4.34007428e-06 -1.24378080e-06 ...  2.28438938e-06
 -3.12186807e-06  1.33607761e-06]


### Zadanie 4
Napisz program, który sumuje zawartość tablicy wypełnionej losowymi wartościa całkowitymi z przedziału od 0 do 100. Sumowanie ma być wykonane pod ochroną szyfrowania homomorficznego.

### Zadanie 5
Oceń wydajność operacji wykonywanych na zaszyfrowanych danych i porównaj z operacjami wykonywamy na wiadomościach jawnych.


### Zadanie 5
Napisz program, który oblicza średnią arytmetyczną losowych wartości rzeczywistych z przedziału od 0,0 do 100.0. Obliczenie ma być wykonane pod ochroną szyfrowania homomorficznego.